# Store music metadata using apple shortcuts
In this small project I was trying to store the metadata about the music that gets shazamed on my macOS
I started by trying to find Shazam database file on my system but that proved to be quit tricky
Note that I tried this on MacOs, it might be different in iOS device
TODO: test on mobile device

## Create a Shortcut
the the shortcut app, click on the plus sign, a new shortcut will be created
<img src="imgs/shazam_create.png" alt="create a new shortcut" width="70%" style="margin:5px;"/>


also note that in the video I recorded I used a loop, bad idea since we can run a shortcut in CLI

I modified the short cut to perform one cycle, it takes no input and outputs a song metadata if a song is found as text, otherwise it return some "No song found"

<img src="imgs/shortcut.png" alt="shows the text file created by the shortcut app" width="80%" style="margin:5px;"/>

In [1]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SHAZAM_TEMPLATE = """
<?xml version="1.0" encoding="UTF-8"?>
<root>

<timestamp>
Date
</timestamp>

<title>
Shazam Media (Title)
</title>

<artist>
Shazam Media (Artist)
</artist>

<isexplicit>
Shazam Media (Is Explicit)
</isexplicit>

<lyricssnippet>
Shazam Media (Lyrics Snippet)
</lyricssnippet>

<lyricsnippetsynced>
Shazam Media (Lyric Snippet Synced)
</lyricsnippetsynced>

<artwork>
Shazam Media (Artwork)
</artwork>

<videourl>
Shazam Media (Video URL)
</videourl>

<shazamurl>
Shazam Media (Shazam URL)
</shazamurl>

<applemusicurl>
Shazam Media (Apple Music URL)
</applemusicurl>

<name>
Shazam Media (Name)
</name>

</root>
"""

In [3]:
outfile = './test.txt'
res = !shortcuts run "shazam_step" --output-path $outfile

In [4]:
res

[]

find the valid tags then find their values

In [5]:
def parse_row(outfile, encoding='utf-8'):
    """
    The parse_row function parses the XML document using lxml parser.
    it uses the `SHAZAM_TEMPLATE` string as map to read the xml data
    ---------------------------------------------------------
    :param outfile: Specify the path to the xml file that will be parsed
    :return: A dictionary with keys as the tag names
        and values as the text content of those tags
    """
    soup = BeautifulSoup(SHAZAM_TEMPLATE, 'xml')
    root = soup.root
    tags = [tag.name  for tag in root.children if tag and tag.name!='root']
    print()
    try:
        with open(outfile, encoding=encoding) as f:
            row = f.read()
    except FileNotFoundError:
        return FileNotFoundError
    # print(row)
    soup = BeautifulSoup(row.replace('\n', ''), 'xml')
    root = soup.select_one('root')
    if root:
        dct = {}
        for tag in tags:
            if not tag:
                continue
            tag_content = root.find(tag)
            if tag_content:
                dct[tag] = tag_content.text
    return dct

In [6]:
parse_row(outfile)

{'timestamp': '10 May 2024 at 15:51',
 'title': 'You Make Me Feel (Radio Edit)',
 'artist': 'Vasscon',
 'isexplicit': 'No',
 'lyricssnippet': '',
 'lyricsnippetsynced': '',
 'artwork': 'Image',
 'videourl': '',
 'shazamurl': 'https://www.shazam.com/track/82175391/you-make-me-feel-radio-edit?co=GB=shortcuts=7277=-4.9489737E-4=231796=2024-05-10T14:51:07.735Z',
 'applemusicurl': '',
 'name': 'Vasscon - You Make Me Feel (Radio Edit)'}

In [7]:
e= Exception
type(e)

type

In [62]:
import os
import pandas as pd
import subprocess

WRITE_FRMT = {
    '.csv': 'to_csv',
    '.xls': 'to_excel',
    '.xlsx': 'to_excel',
    '.json': 'to_json',
    '.html': 'to_html',
    '.sql': 'to_sql',
    '.parquet': 'to_parquet',
    '.feather': 'to_feather',
    '.h5': 'to_hdf',
    '.hdf': 'to_hdf',
    '.dta': 'to_stata',
    '.sas7bdat': 'to_sas',
}

READ_FRMT = {
    '.csv': 'read_csv',
    '.xls': 'read_excel',
    '.xlsx': 'read_excel',
    '.json': 'read_json',
    '.html': 'read_html',
    '.sql': 'read_sql',
    '.parquet': 'read_parquet',
    '.feather': 'read_feather',
    '.h5': 'read_hdf',
    '.hdf': 'read_hdf',
    '.dta': 'read_stata',
    '.sas7bdat': 'read_sas',
}


def detect_file_type(file_path):

    # Run the file command and capture the output
    """
    The detect_file_type function takes a file path as input and returns the
    file type. The function uses the Linux 'file' command to determine the file
    type. If you are using Windows, you will need to install Cygwin or WSL in order
    to use this function.
    -----------------------------------------------
    :param file_path: Specify the file to be checked
    :return: A string containing the file type
    """
    result = subprocess.run(
        ['file', '--mime-type', file_path], capture_output=True, text=True)

    # Extract the file type from the output
    file_type = result.stdout.strip().split(': ')[-1]

    return file_type


def read_frmt(db_file, args={}):
  
    """
    The read_frmt function reads in a data file
    and returns the contents as a pandas DataFrame.
    -------------------------------------------------------
    :param db_file: Pass the name of the file to be read
    :param args: Pass in the arguments from the command line
    :return: A dataframe

    """
    print(db_file)
    _, ext = os.path.splitext(db_file)

    rgx = '|'.join(map(lambda c: c.strip('.'), READ_FRMT.keys()))
    rgx = rf'({rgx})'
    ft = detect_file_type(db_file)
    ft = re.findall(rgx, ft)
    if len(ft) > 0:
        frmt = f'.{ft[0]}'
    else:
        frmt = ext
    return read(db_file, frmt,args)

def read(db_file, frmt, args={}):
    """
    The read function reads a database file and returns
    a pandas dataframe.
    -------------------------------------------
    :param db_file: Specify the file to be read
    :param frmt: specify the format of the file being read
    :param args: Specify the arguments to use when reading the file
    :return: A pandas dataframe with the contents of the file
    """
    # print(frmt)
    method = READ_FRMT.get(frmt)
    if method:
        read_method = getattr(pd, method)
        df = read_method(db_file, **args)
        return df


In [65]:
read_frmt('shazam.csv')

shazam.csv
.csv


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,timestamp,title,artist,isexplicit,lyricssnippet,lyricsnippetsynced,artwork,videourl,shazamurl,applemusicurl,name
0,0,0.0,0.0,0.0,0.0,9 May 2024 at 17:12,In Your Time,LEE SUHYUN,No,차갑게 남겨진 이야기그때의 모든 게 난 선명해동화같은 시간 그 속에 난남겨짐만 그 ...,이렇게 다시 너를 만나지워버린 그때의 너를 찾아아팠던 시간 지나내가 널 붙잡아아직 ...,Image,NaN,https://www.shazam.com/track/529747941/in-your...,https://music.apple.com/gb/album/in-your-time/...,LEE SUHYUN - In Your Time
1,1,1.0,1.0,1.0,NaN,9 May 2024 at 17:39,Perhaps Love,HowL J,No,언제였던 건지 기억나지 않아자꾸 내 머리가 너로 어지럽던 시절한두 번씩 떠오르던 생...,자꾸 내 머리가 너로 어지럽던 시절한두 번씩 떠오르던 생각자꾸 늘어가서 조금 당황스...,Image,NaN,https://www.shazam.com/track/44655269/perhaps-...,https://music.apple.com/gb/album/perhaps-love/...,HowL J - Perhaps Love
2,2,2.0,NaN,NaN,NaN,10 May 2024 at 12:04,The First Snow (Instrumental),Jung Joonil,No,NaN,NaN,Image,NaN,https://www.shazam.com/track/339059877/the-fir...,https://music.apple.com/gb/album/the-first-sno...,Jung Joonil - The First Snow (Instrumental)
3,3,NaN,NaN,NaN,NaN,10 May 2024 at 12:18,You Are So Beautiful,Eddy Kim,No,자꾸 물어보는 너늘 확인하고픈 너같은 말 해줘도매일 말해줘도왜 계속 물어입술이 말라...,정말 힌트도 없는지너 정말 이쁘다 이쁘다 이쁘다니까왜 내 말 믿지 않는 건데왜 말하...,Image,NaN,https://www.shazam.com/track/338367689/you-are...,https://music.apple.com/gb/album/you-are-so-be...,Eddy Kim - You Are So Beautiful


In [64]:
df

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,timestamp,title,artist,isexplicit,lyricssnippet,lyricsnippetsynced,artwork,videourl,shazamurl,applemusicurl,name
0,0,0.0,0.0,0.0,0.0,9 May 2024 at 17:12,In Your Time,LEE SUHYUN,No,차갑게 남겨진 이야기그때의 모든 게 난 선명해동화같은 시간 그 속에 난남겨짐만 그 ...,이렇게 다시 너를 만나지워버린 그때의 너를 찾아아팠던 시간 지나내가 널 붙잡아아직 ...,Image,NaN,https://www.shazam.com/track/529747941/in-your...,https://music.apple.com/gb/album/in-your-time/...,LEE SUHYUN - In Your Time
1,1,1.0,1.0,1.0,NaN,9 May 2024 at 17:39,Perhaps Love,HowL J,No,언제였던 건지 기억나지 않아자꾸 내 머리가 너로 어지럽던 시절한두 번씩 떠오르던 생...,자꾸 내 머리가 너로 어지럽던 시절한두 번씩 떠오르던 생각자꾸 늘어가서 조금 당황스...,Image,NaN,https://www.shazam.com/track/44655269/perhaps-...,https://music.apple.com/gb/album/perhaps-love/...,HowL J - Perhaps Love
2,2,2.0,NaN,NaN,NaN,10 May 2024 at 12:04,The First Snow (Instrumental),Jung Joonil,No,NaN,NaN,Image,NaN,https://www.shazam.com/track/339059877/the-fir...,https://music.apple.com/gb/album/the-first-sno...,Jung Joonil - The First Snow (Instrumental)
3,3,NaN,NaN,NaN,NaN,10 May 2024 at 12:18,You Are So Beautiful,Eddy Kim,No,자꾸 물어보는 너늘 확인하고픈 너같은 말 해줘도매일 말해줘도왜 계속 물어입술이 말라...,정말 힌트도 없는지너 정말 이쁘다 이쁘다 이쁘다니까왜 내 말 믿지 않는 건데왜 말하...,Image,NaN,https://www.shazam.com/track/338367689/you-are...,https://music.apple.com/gb/album/you-are-so-be...,Eddy Kim - You Are So Beautiful


In [15]:


# Example usage:
file_path = 'file.json'
file_type = detect_file_type(file_path)
print("File Type:", file_type)

File Type: text/csv


In [22]:
file_type

'text/csv'

In [39]:
rgx = '|'.join(map(lambda c: c.strip('.'), READ_FRMT.keys()))
rgx = rf'({rgx})'

len(re.findall(rgx, file_type)) > 0


True

In [20]:
rgx

'csv|xls|xlsx|json|html|sql|parquet|feather|h5|hdf|dta|sas7bdat'